## Image Classification

In [1]:
import os
import json
import math
import requests
import subprocess
import numpy as np
import argparse
import time
import pandas as pd

from shapely.geometry import box
from shapely.geometry import Polygon
from osgeo import gdal, gdal_array, ogr, osr, gdalconst
from owslib.wms import WebMapService
from sklearn.ensemble import RandomForestClassifier

import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np
from shapely.geometry import mapping

ogr.UseExceptions()

In [6]:
   
image=r"https://minsait-geospatial.s3.eu-west-3.amazonaws.com/data/EO_Analytics/ImageClassification/class_test.tif"
shape=r"https://minsait-geospatial.s3.eu-west-3.amazonaws.com/data/EO_Analytics/ImageClassification/class_mfe.geojson"
raster_training_file= r"E:\class_rf.tif"
outFileName=r"E:\class_predic_rf.tif"

fieldClass= "O1"
outFileName = None, 
debug=True
rasterization=False

# Timing

# Open the dataset from the file
dataset = ogr.Open(shape)
# Make sure the dataset exists -- it would be None if we couldn't open it
if not dataset:
    print('Error: could not open dataset')
### Let's get the driver from this file
driver = dataset.GetDriver()
if debug:
    print('Dataset driver is: {n}\n'.format(n=driver.name))

### How many layers are contained in this Shapefile?
layer_count = dataset.GetLayerCount()
if debug:
    print('The shapefile has {n} layer(s)\n'.format(n=layer_count))

### What is the name of the 1 layer?
layer = dataset.GetLayerByIndex(0)
if debug:
    print('The layer is named: {n}\n'.format(n=layer.GetName()))

### What is the layer's geometry? is it a point? a polyline? a polygon?
# First read in the geometry - but this is the enumerated type's value
geometry = layer.GetGeomType()

# So we need to translate it to the name of the enum
geometry_name = ogr.GeometryTypeToName(geometry)
if debug:
    print("The layer's geometry is: {geom}\n".format(geom=geometry_name))

### What is the layer's projection?
# Get the spatial reference
spatial_ref = layer.GetSpatialRef()


### How many features are in the layer?
feature_count = layer.GetFeatureCount()
if debug:
    print('Layer has {n} features\n'.format(n=feature_count))

### How many fields are in the shapefile, and what are their names?
# First we need to capture the layer definition
defn = layer.GetLayerDefn()

if debug:
    # How many fields
    field_count = defn.GetFieldCount()
    print('Layer has {n} fields'.format(n=field_count))
    # What are their names?
    print('Their names are: ')
    for i in range(field_count):
        field_defn = defn.GetFieldDefn(i)
        print('\t{name} - {datatype}'.format(name=field_defn.GetName(),
                                             datatype=field_defn.GetTypeName()))

# gdal_rasterize

# First we will open our raster image, to understand how we will want to rasterize our vector
raster_ds = gdal.Open(image, gdal.GA_ReadOnly)

# Fetch number of rows and columns
ncol = raster_ds.RasterXSize
nrow = raster_ds.RasterYSize

# Fetch projection and extent
proj = raster_ds.GetProjectionRef()
ext = raster_ds.GetGeoTransform()

# raster_ds = None

# Create the raster dataset
memory_driver = gdal.GetDriverByName('GTiff')
out_raster_ds = memory_driver.Create(raster_training_file, ncol, nrow, 1, gdal.GDT_Byte)

# Set the ROI image's projection and extent to our input raster's projection and extent
out_raster_ds.SetProjection(proj)
out_raster_ds.SetGeoTransform(ext)

# Fill our output band with the 0 blank, no class label, value
b = out_raster_ds.GetRasterBand(1)
b.Fill(0)

# Rasterize the shapefile layer to our new dataset
# The shapefile is already projected in the same projection

status = gdal.RasterizeLayer(out_raster_ds,  # output to our new dataset
                             [1],  # output to our new dataset's first band
                             layer,  # rasterize this layer
                             None, None,  # don't worry about transformations since we're in same projection
                             [0],  # burn value 0
                             ['ALL_TOUCHED=TRUE',  # rasterize all pixels touched by polygons
                              'ATTRIBUTE=%s' % (fieldClass)]  # put raster values according to the 'id' field values
                             )

##ruta_bin = "/usr/bin/"
# ruta_bin = "C:\\PROGRA~1\\GDAL\\"
# cmd = ruta_bin + 'gdal_rasterize.exe -b 1 -at -a ID_COBERTU -l sample_burela_pri %s %s' % (shape, raster_training_file)
# print cmd
# lanzar_proceso(cmd)
# gdal.Rasterize(out_raster_ds, dataset)

# Close dataset
out_raster_ds = None
if debug:
    if status != 0:
        print("I don't think it worked...")
    else:
        print("Success")

# First we will open our raster image, to understand how we will want to rasterize our vector
# raster_ds = gdal.Open(raster_file, gdal.GA_ReadOnly)
# Fetch number of rows and columns
# ncol = raster_ds.RasterXSize
# nrow = raster_ds.RasterYSize

# Fetch projection and extent
# proj = raster_ds.GetProjectionRef()
# ext = raster_ds.GetGeoTransform()
# raster_ds = None

# Create the raster dataset
memory_driver = gdal.GetDriverByName('GTiff')

# img_ds = gdal.Open(image, gdal.GA_ReadOnly)
roi_ds = gdal.Open(raster_training_file, gdal.GA_ReadOnly)

img = np.zeros((raster_ds.RasterYSize, raster_ds.RasterXSize, raster_ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(raster_ds.GetRasterBand(1).DataType))
for b in range(img.shape[2]):
    img[:, :, b] = raster_ds.GetRasterBand(b + 1).ReadAsArray()

roi = roi_ds.GetRasterBand(1).ReadAsArray().astype(np.uint8)

# Display them
# plt.subplot(121)
# plt.imshow(img[:, :, 3], cmap=plt.cm.Greys_r)
# plt.title('B3')
#
# plt.subplot(122)
# plt.imshow(roi, cmap=plt.cm.Spectral)
# plt.title('ROI Training Data')
# plt.show()

# Find how many non-zero entries we have -- i.e. how many training data samples?
n_samples = (roi > 0).sum()
if debug:
    print('We have {n} samples'.format(n=n_samples))

# What are our classification labels?

# _________________________________________________________________ medida temporal
#roi = np.where(roi == 57, 313, roi)
# ________________________________________________________________________________

labels = np.unique(roi[roi > 0])
print('The training data include {n} classes: {classes}'.format(n=labels.size, classes=labels))
# We will need a "X" matrix containing our features, and a "y" array containing our labels
#     These will have n_samples rows
#     In other languages we would need to allocate these and them loop to fill them, but NumPy can be faster

X = img[roi > 0]  # include 8th band, which is Fmask, for now
# X = img
y = roi[roi > 0]
# y = roi
if debug:
    print('Our X matrix is sized: {sz}'.format(sz=X.shape))
    print('Our y array is sized: {sz}'.format(sz=y.shape))

# Mask out clouds, cloud shadows, and snow using Fmask
# clear = X[:, 7] <= 1
#
# X = X[clear, :7]  # we can ditch the Fmask band now
# y = y[clear]
if debug:
    print('After masking, our X matrix is sized: {sz}'.format(sz=X.shape))
    print('After masking, our y array is sized: {sz}'.format(sz=y.shape))

# Initialize our model with 500 trees
# rf = sk.ensemble.RandomForestClassifier(n_estimators=500, oob_score=True)
rf = RandomForestClassifier(n_estimators=500, oob_score=True)
# Fit our model to training data
rf = rf.fit(X, y)

# Random Forest diagnostics

if debug:
    print('Our OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100))

bandsCount = raster_ds.RasterCount
bands = np.arange(bandsCount)
# feature importance scores
for b, imp in zip(bands, rf.feature_importances_):
    print('Band {b} importance: {imp}'.format(b=b, imp=imp))
# Crosstabulation to see the class confusion

# Setup a dataframe -- just like R
df = pd.DataFrame()
df['truth'] = y
df['predict'] = rf.predict(X)

# Cross-tabulate predictions
if debug:
    print(pd.crosstab(df['truth'], df['predict'], margins=True))

# Predicting the rest of the image

# Take our full image, ignore the Fmask band, and reshape into long 2d array (nrow * ncol, nband) for classification
# new_shape = (img.shape[0] * img.shape[1], img.shape[2] - 1)
new_shape = (img.shape[0] * img.shape[1], img.shape[2])

img_as_array = img[:, :, :img.shape[2]].reshape(new_shape)
# img_as_array = img[:, :, :7].reshape(new_shape)
if debug:
    print('Reshaped from {o} to {n}'.format(o=img.shape, n=img_as_array.shape))

# Now predict for each pixel
start = time.time()
class_prediction = rf.predict(img_as_array)
end = time.time()
if debug:
    print ('Class prediction took %s seconds' % (end - start))

# Reshape our classification map
class_prediction = class_prediction.reshape(img[:, :, 0].shape)

Dataset driver is: GeoJSON

The shapefile has 1 layer(s)

The layer is named: class_mfe

The layer's geometry is: Unknown (any)

Layer has 175 features

Layer has 18 fields
Their names are: 
	PROV_MFE_1 - String
	USOS_GENER - String
	TIPESTR_DS - String
	FORM_ARB_D - String
	TIPO_BOSQU - String
	SP1_DS - String
	O1 - Integer
	E1_DS - String
	SP2_DS - String
	O2 - Integer
	E2_DS - String
	SP3_DS - String
	O3 - Integer
	E3_DS - String
	FCCTOT - Integer
	FCCARB - Integer
	DISTRIB_DS - String
	SUPERFICIE - Real
Success
We have 187736 samples
The training data include 7 classes: [30 40 50 60 70 80 90]
Our X matrix is sized: (187736, 3)
Our y array is sized: (187736,)
After masking, our X matrix is sized: (187736, 3)
After masking, our y array is sized: (187736,)
Our OOB prediction of accuracy is: 36.73083478927856%
Band 0 importance: 0.2914518737974769
Band 1 importance: 0.3341458741686241
Band 2 importance: 0.3744022520338991
predict    30     40     50     60     70    80    90     All
tr

In [5]:
cols = img.shape[0]
rows = img.shape[1]
driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create(outFileName, rows, cols, 1, gdal.GDT_UInt16)
outdata.SetGeoTransform(raster_ds.GetGeoTransform())  ##sets same geotransform as input
outdata.SetProjection(raster_ds.GetProjection())  ##sets same projection as input
outdata.GetRasterBand(1).WriteArray(class_prediction)
outdata.GetRasterBand(1).SetNoDataValue(10000)  ##if you want these values transparent
outdata.FlushCache()  ##saves to disk!!
outdata = None